In [ ]:
train_x = [14, 17, 20]
train_y = [220, 240, 300]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import normal

In [ ]:
slope = 10.5
y_intercept = 20.3

print(f"(slope: {slope}, y_intercept: {y_intercept})")

In [ ]:
def visualize_linear_regression(ax, title, x, y, train_x, train_y):
    ax.scatter(train_x, train_y, c="#41B06E")
    ax.set(ylim=[0, 500], xlim=[0, 30])
    ax.plot(x, y)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(title)

In [ ]:
def linear_regression(x, slope, y_intercept):
    return x * slope + y_intercept

In [ ]:
x = np.linspace(0., 300, 5)
y = linear_regression(x, slope, y_intercept)

fig, ax = plt.subplots()
visualize_linear_regression(ax, 'train_data', x, y, train_x, train_y)

In [ ]:
def sse(y, y_hat):
    return (y - y_hat)**2

In [ ]:
def derivative_slope(f, x, slope, y_intercept, y, y_hat, dx = 0.000001):
    return (f(y, y_hat(x, slope, y_intercept)) - f(y, y_hat(x, slope + dx, y_intercept)))/(slope - (slope + dx))

In [ ]:
def derivative_y_intercept(f, x, slope, y_intercept, y, y_hat, dx = 0.000001):
    return (f(y, y_hat(x, slope, y_intercept)) - f(y, y_hat(x, slope, y_intercept + dx)))/(y_intercept - ( y_intercept + dx))

In [ ]:
def step_size(f, train_x, slope, train_y, learning_rate = 0.001):
    d_slope = 0.
    d_y_intercept = 0.

    train_data = zip(train_x, train_y)
    
    for x, y in train_data:
        d_slope += derivative_slope(f, x, slope, y_intercept, y, linear_regression)
        d_y_intercept += derivative_y_intercept(f, x, slope, y_intercept, y, linear_regression)
    
    return (learning_rate * d_slope, learning_rate * d_y_intercept)

In [ ]:
slope_stack = []
y_intercept_stack = []

slope_step_size = 0.
y_intercept_step_size = 0.

batch_size = 1
offset = 0
index = 0

for i in range(100):
    new_slope_step_size, new_y_intercept_step_size = step_size(
        sse, 
        train_x[offset:(index - 1)*batch_size], 
        slope, 
        train_y[offset:(index - 1)*batch_size],
        learning_rate = 0.001,
    )

    slope_step_size += new_slope_step_size
    y_intercept_step_size += new_y_intercept_step_size

    slope_new = slope - slope_step_size
    y_intercept_new = y_intercept - y_intercept_step_size

    slope = slope_new
    y_intercept = y_intercept_new

    slope_stack.append(slope_new)
    y_intercept_stack.append(y_intercept_new)

    error_dot_product = np.dot(np.abs(slope_step_size), np.abs(y_intercept_step_size))
    if error_dot_product <= 0.000001: break

    offset = index*batch_size
    if index == len(train_x):
        index = 0 
    else:
        index += 1

In [ ]:
y_updated = linear_regression(x, slope, y_intercept)

fig, (ax_1, ax_2)  = plt.subplots(nrows=1, ncols=2, figsize=(5, 2.5))
visualize_linear_regression(ax_1, 'initialization', x , y, train_x, train_y)
visualize_linear_regression(ax_2, 'optimized', x, y_updated, train_x, train_y)
fig.tight_layout()

In [ ]:
import matplotlib.animation as animation

In [ ]:
fig, ax = plt.subplots()

scat = ax.scatter(train_x, train_y, c="#41B06E")
line = ax.plot(x, slope_stack[0]*x + y_intercept_stack[0])[0]

ax.set(ylim=[0, 500], xlim=[0, 30], xlabel='x', ylabel='y')

def update(frame):
    line.set_xdata(x)
    line.set_ydata(slope_stack[frame]*x + y_intercept_stack[frame])
    ax.set_title(f"y = {slope_stack[frame]:.6f}x + {y_intercept_stack[frame]:.6f}")
    return (scat, line)

frames = len(slope_stack)
ani = animation.FuncAnimation(fig=fig, func=update, frames=frames, interval=200)

from IPython.display import HTML
HTML(ani.to_jshtml())